In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split


from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization


import pandas as pd
import numpy as np

import pickle

In [ ]:
features_columns = [
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",
    "RADIACAO GLOBAL (KJ/m²)",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)",
    "UMIDADE RELATIVA DO AR, HORARIA (%)",
    "VENTO, VELOCIDADE HORARIA (m/s)"
]

target_column = ["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"]

In [ ]:
X = pd.read_csv("/kaggle/input/training-dataset-random-forest/step_5_df.csv", usecols=features_columns)
y = pd.read_csv("/kaggle/input/training-dataset-random-forest/step_5_df.csv", usecols=target_column)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [ ]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.to_numpy().reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.to_numpy().reshape(-1, 1))

In [ ]:
X, y, X_train, X_test, y_train, y_test = 0, 0, 0, 0, 0, 0

In [ ]:
model = Sequential([
    Input(shape=(6,)),  # Define explicitamente a forma da entrada
    Dense(64, activation='relu'),
    Dropout(0.2),  # Apaga 20% dos neurônios durante o treinamento
    BatchNormalization(),
    Dense(32, activation='relu'),
    Dropout(0.2),
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

checkpoint = ModelCheckpoint(
    filepath='/kaggle/working/modelo_checkpoint.keras',  # Nome do arquivo a ser salvo
    monitor='val_loss',               # Métrica a ser monitorada
    save_best_only=False,             # Salvar a cada época (não apenas o melhor)
    save_weights_only=False,          # Salvar modelo completo (estrutura + pesos)
    verbose=1                         # Exibir informações no terminal
)

In [ ]:
try:
    model = load_model('/kaggle/working/modelo_checkpoint.keras')    
except Exception as e:
    print(e)


In [14]:
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

early_stopping = EarlyStopping(monitor='val_mse', patience=2, restore_best_weights=True)
history = model.fit(X_train_scaled, y_train_scaled, epochs=20, batch_size=128, validation_split=0.2, callbacks=[early_stopping, checkpoint])


149718/149730 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.6526e-05 - mae: 0.0026 - mse: 8.6526e-05
Epoch 2: saving model to /kaggle/working/modelo_checkpoint.keras
149730/149730 ━━━━━━━━━━━━━━━━━━━━ 613s 4ms/step - loss: 8.6526e-05 - mae: 0.0026 - mse: 8.6526e-05 - val_loss: 8.8071e-05 - val_mae: 0.0019 - val_mse: 8.8071e-05
Epoch 3/20
149723/149730 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.5944e-05 - mae: 0.0026 - mse: 8.5944e-05
Epoch 3: saving model to /kaggle/working/modelo_checkpoint.keras
149730/149730 ━━━━━━━━━━━━━━━━━━━━ 641s 4ms/step - loss: 8.5944e-05 - mae: 0.0026 - mse: 8.5944e-05 - val_loss: 8.7263e-05 - val_mae: 0.0020 - val_mse: 8.7263e-05


In [ ]:
with open('historico.pkl', 'wb') as f:
    pickle.dump(history.history, f)

In [15]:
y_pred_scaled = model.predict(X_test_scaled)

y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test = scaler_y.inverse_transform(y_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse:.2f}")
print(f"R²: {r2:.2f}")

499098/499098 ━━━━━━━━━━━━━━━━━━━━ 751s 2ms/step
MSE: 1.64
R²: 0.05
